In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [16]:
joke=pd.read_csv('jokes.csv')
joke.set_index('joke_id',inplace=True)
joke.head(5)

,joke_text
joke_id,
1,Q. What's O. J. Simpson's web address? A. Slas...
2,How many feminists does it take to screw in a ...
3,Q. Did you hear about the dyslexic devil worsh...
4,They asked the Japanese visitor if they have e...
5,Q: What did the blind person say when given so...


In [17]:
t=pd.read_csv('train.csv')
t.set_index('joke_id',inplace=True)
t.head(1)

,id,user_id,Rating
joke_id,,,
110,31030_110,31030,2.75


In [18]:
t['joke']=joke['joke_text']
t['len']=joke['joke_text'].apply(len)

In [21]:
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
110,31030_110,31030,2.750,"Judy was having trouble with her computer, so ...",562
109,16144_109,16144,5.094,One day the first grade teacher was reading th...,541
6,23098_6,23098,-6.438,Q. What is orange and sounds like a parrot? A....,56
86,14273_86,14273,4.406,Two attorneys went into a diner and ordered tw...,329
134,18419_134,18419,9.375,A man is driving in the country one evening wh...,759


## Since jokes ratings are varying b/w positive and negative no., So we are going to consider jokes that have a rating above 0 as 1(funny) and rest as 0(not funny).

In [22]:
def rate(x):
    if x>=0:
        return 1
    else:
        return 0
p=t.Rating.map(rate)
t.Rating=p
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
110,31030_110,31030,1,"Judy was having trouble with her computer, so ...",562
109,16144_109,16144,1,One day the first grade teacher was reading th...,541
6,23098_6,23098,0,Q. What is orange and sounds like a parrot? A....,56
86,14273_86,14273,1,Two attorneys went into a diner and ordered tw...,329
134,18419_134,18419,1,A man is driving in the country one evening wh...,759


In [23]:
t=t.sort_index(kind='quicksort',axis=0)
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
1,74_1,74,0,Q. What's O. J. Simpson's web address? A. Slas...,88
1,490_1,490,1,Q. What's O. J. Simpson's web address? A. Slas...,88
1,315_1,315,0,Q. What's O. J. Simpson's web address? A. Slas...,88
1,337_1,337,1,Q. What's O. J. Simpson's web address? A. Slas...,88
1,457_1,457,1,Q. What's O. J. Simpson's web address? A. Slas...,88


# Data Cleaning

In [24]:
t=t.drop_duplicates(subset={'joke','len'},keep='first')
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
1,74_1,74,0,Q. What's O. J. Simpson's web address? A. Slas...,88
2,31674_2,31674,1,How many feminists does it take to screw in a ...,75
3,37340_3,37340,1,Q. Did you hear about the dyslexic devil worsh...,81
4,39978_4,39978,1,They asked the Japanese visitor if they have e...,99
5,5568_5,5568,0,Q: What did the blind person say when given so...,84


In [25]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english')) #set of stopwords
sno = SnowballStemmer('english') #initialising the snowball stemmer

def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|:)|(|\|/]',r' ',cleaned)
    cleaned=cleaned.lower()
    return  cleaned


In [28]:
final_string=[]
for sent in t['joke']:
    filtered_sentence=[]
    for cleaned_words in cleanpunc(sent).split():
            if(len(cleaned_words)>2):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
        
    str1 = b" ".join(filtered_sentence) 
    final_string.append(str1)

In [29]:
final_string


[b'what simpson web address slash slash backslash slash slash escap',
 b'mani feminist take screw light bulb that funni',
 b'hear dyslex devil worship sold soul santa',
 b'ask japanes visitor elect countri everi morn answer',
 b'blind person say given matzah hell wrote',
 b'orang sound like parrot carrot',
 b'mani men take screw light bulb one men screw anyth',
 b'dog walk western union ask clerk send telegram fill form write telegram wish send bow wow wow bow wow wow clerk say add anoth bow wow price dog respond wouldnt sound littl silli',
 b'person speak three languag call trilingu person speak two languag call bilingu call person speak one languag american',
 b'what differ macintosh etch-a-sketch dont shake mac clear screen',
 b'what differ use tire 365 use condom one goodyear other great year',
 b'duck walk pharmaci ask condom pharmacist ask would like stick bill duck say kind duck think',
 b'australian word boomerang wont come back stick',
 b'get run parakeet lawnmow shred tweet',

In [81]:
cl=[]
for i in t['joke']:
    c=[]
    for j in i.split():
        if len(j)>2:
            if(j.lower() not in stop):
                s=(sno.stem(j.lower())).encode('utf8')
                c.append(s)
    st=b" ".join(c)
    cl.append(st)
t['cjoke']=cl
t.head()

,Rating,joke,len,cjoke
joke_id,,,,
1,1,q whats o j simpsons web address a slash ...,88,b'what simpson web address slash slash backsla...
2,1,how many feminists does it take to screw in a ...,75,b'mani feminist take screw light bulb that funni'
3,1,q did you hear about the dyslexic devil worsh...,81,b'hear dyslex devil worship sold soul santa'
4,0,they asked the japanese visitor if they have e...,99,b'ask japanes visitor elect countri everi morn...
5,1,q: what did the blind person say when given so...,84,b'blind person say given matzah hell wrote'


In [88]:
t=t.drop('joke',axis=1)
t['len']=t['cjoke'].apply(len)
t.head()

,Rating,len,cjoke
joke_id,,,
1,1,64,b'what simpson web address slash slash backsla...
2,1,46,b'mani feminist take screw light bulb that funni'
3,1,41,b'hear dyslex devil worship sold soul santa'
4,0,51,b'ask japanes visitor elect countri everi morn...
5,1,40,b'blind person say given matzah hell wrote'


In [89]:
t.to_csv('finaljoke')


# This is our final data after cleaning and rearrangement

In [3]:
f=pd.read_csv('finaljoke',index_col='joke_id')
f.head()

,Rating,len,cjoke
joke_id,,,
1,1,64,b'what simpson web address slash slash backsla...
2,1,46,b'mani feminist take screw light bulb that funni'
3,1,41,b'hear dyslex devil worship sold soul santa'
4,0,51,b'ask japanes visitor elect countri everi morn...
5,1,40,b'blind person say given matzah hell wrote'


# We are using Bag of Words technique on cjoke text 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X=f.cjoke
y=f.Rating
X=cv.fit_transform(X)

# Splitting the data into train and test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)

# We are using Naive Bayes classifier

In [8]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)
predictions = nb.predict(X_test)

# Checking the accuracy

In [9]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[ 8  6]
 [12 16]]


             precision    recall  f1-score   support

          0       0.40      0.57      0.47        14
          1       0.73      0.57      0.64        28

avg / total       0.62      0.57      0.58        42



## Our accuracy is around 60% which is not good. Now let's try using TfidTransformer to add weight to the vector that we got from bag of words and we will standardised it by dividing every element with their std deviation. We are going to use pipeline feature.

In [11]:
from sklearn.feature_extraction.text import  TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),
    ('standscaler',StandardScaler(with_mean=False)),# integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
X = f.cjoke
y = f.Rating
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...=False, with_std=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [12]:
predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 5  9]
 [ 3 25]]
             precision    recall  f1-score   support

          0       0.62      0.36      0.45        14
          1       0.74      0.89      0.81        28

avg / total       0.70      0.71      0.69        42



Now we are getting around 70%. It is still not enough but keep in mind that we have a small dataset and you can try another techniques like word2vec to get better results.